# 처음 해보는 CUDA Programming

가볍게 배열을 복사하는 프로그램을 짜봅시다

### Copying an array in C

메모리에 있는 데이터를 복사할 때 우리는 보통 API를 이용해서 처리합니다.

In [ ]:
%%file vector_copy.cc
#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define BUF_SIZE 256

int main() {
    float* p_x = (float*)malloc(BUF_SIZE * sizeof(float));
    float* p_y = (float*)malloc(BUF_SIZE * sizeof(float));
    
    for (int i = 0; i < BUF_SIZE; i++) {
        p_x[i] = i * 1.5f;
    }

    // Write vector copy code
    memcpy(p_y, p_x, BUF_SIZE * sizeof(float));
    
    float sum = 0.f;
    for (int i = 0; i < BUF_SIZE; i++) {
        sum += p_y[i] - p_x[i];
    }
    printf("Diff. Sum: %f", sum);
    
    free(p_x);
    free(p_y);
}

In [ ]:
!make vector_copy && ./vector_copy

# Writing Iterative Copying Code

위에서 사용한 memcpy는 사실 내부적으로는 순차적으로 복사하는 과정을 거치게 됩니다.

memcpy 명령을 그대로 둔 상태로 복사를 해 보도록 하겠습니다.

In [ ]:
%%file buffer_copy_iter.cc
#include <stdio.h>
#include <stdlib.h>

#define BUF_SIZE 256

void memcpy(float *out, float *in, size_t length) {
    for (int i = 0; i < length / (int)sizeof(float); i++) {
        out[i] = in[i];
    }
}

int main() {
    float* p_x = (float*)malloc(BUF_SIZE * sizeof(float));
    float* p_y = (float*)malloc(BUF_SIZE * sizeof(float));
    
    for (int i = 0; i < BUF_SIZE; i++) {
        p_x[i] = i * 1.5f;
    }

    // Write vector copy code
    memcpy(p_y, p_x, BUF_SIZE * sizeof(float));
    
    float sum = 0.f;
    for (int i = 0; i < BUF_SIZE; i++) {
        sum += p_y[i] - p_x[i];
    }
    printf("Diff. Sum: %f", sum);
    
    free(p_x);
    free(p_y);
}

In [ ]:
!make buffer_copy_iter && ./buffer_copy_iter

## CUDA Buffer Allocation

이제 CUDA 코드 작성을 시작해 보도록 하겠습니다. 먼저 시작할 내용은 GPU에서 사용할 메모리를 할당하고 해제하는 것입니다.

In [ ]:
%%file buffer_alloc.cc
#include <stdio.h>
#include <stdlib.h>

#define BUF_SIZE 256

void memcpy(float *out, float *in, size_t length) {
    for (int i = 0; i < length / (int)sizeof(float); i++) {
        out[i] = in[i];
    }
}

int main() {
    float* p_x = (float*)malloc(BUF_SIZE * sizeof(float));
    float* p_y = (float*)malloc(BUF_SIZE * sizeof(float));
    
    // Step 1. CUDA Buffer allocation
    cudaMalloc()
    
    for (int i = 0; i < BUF_SIZE; i++) {
        p_x[i] = i * 1.5f;
    }

    // Write vector copy code
    memcpy(p_y, p_x, BUF_SIZE * sizeof(float));
    
    float sum = 0.f;
    for (int i = 0; i < BUF_SIZE; i++) {
        sum += p_y[i] - p_x[i];
    }
    printf("Diff. Sum: %f", sum);
    
    // Step 2. CUDA buffer free
    cudaFree()
    
    free(p_x);
    free(p_y);
}

In [ ]:
!make buffer_alloc && ./buffer_alloc

## Copying Host data to GPU memory

CUDA 메모리를 할당했으니 CUDA 메모리에 복사해 보도록 하겠습니다.

현재로서는 아무것도 하지는 않지만, GPU가 처리할 데이터를 준비하는 과정이되는 중요한 과정입니다.

In [ ]:
%%file cuda_memcpy.cu
#include <stdio.h>
#include <stdlib.h>

#define BUF_SIZE 256

void memcpy(float *out, float *in, size_t length) {
    for (int i = 0; i < length / (int)sizeof(float); i++) {
        out[i] = in[i];
    }
}

int main() {
    float* p_x = (float*)malloc(BUF_SIZE * sizeof(float));
    float* p_y = (float*)malloc(BUF_SIZE * sizeof(float));
    
    // Step 1. CUDA Buffer allocation
    float *d_x, *d_y;
    cudaMalloc((void**)&d_x, BUF_SIZE * sizeof(float));
    cudaMalloc((void**)&d_y, BUF_SIZE * sizeof(float));
               
    for (int i = 0; i < BUF_SIZE; i++) {
        p_x[i] = i * 1.5f;
    }
    
    // Step 3. CUDA memcpy (Host -> Device)
    cudaMemcpy();

    // Write vector copy code
    memcpy(p_y, p_x, BUF_SIZE * sizeof(float));
    
    // Step 4. CUDA memcpy (Device -> Host)
    cudaMemcpy()
    
    float sum = 0.f;
    for (int i = 0; i < BUF_SIZE; i++) {
        sum += p_y[i] - p_x[i];
    }
    printf("Diff. Sum: %f", sum);
    
    // Step 2. CUDA buffer free
    cudaFree(d_x);
    cudaFree(d_y);
    
    free(p_x);
    free(p_y);
}

In [ ]:
!make cuda_memcpy && ./cuda_memcpy

## Writing Kernel Code & Call

이제 GPU 함수인 Kernel 코드를 작성해보고 CPU 코드와 비교해 보도록 하겠습니다.

In [ ]:
%%file kernel_call.cu
#include <stdio.h>
#include <stdlib.h>

#define BUF_SIZE 256

void memcpy(float *out, float *in, size_t length) {
    for (int i = 0; i < length / (int)sizeof(float); i++) {
        out[i] = in[i];
    }
}

// Step 5. Writing Kernel Code

int main() {
    float* p_x = (float*)malloc(BUF_SIZE * sizeof(float));
    float* p_y = (float*)malloc(BUF_SIZE * sizeof(float));
    
    // Step 1. CUDA Buffer allocation
    float *d_x, *d_y;
    cudaMalloc((void**)&d_x, BUF_SIZE * sizeof(float));
    cudaMalloc((void**)&d_y, BUF_SIZE * sizeof(float));
               
    for (int i = 0; i < BUF_SIZE; i++) {
        p_x[i] = i * 1.5f;
    }
    
    // Step 3. CUDA memcpy (Host -> Device)
    cudaMemcpy(d_x, p_x, BUF_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    // Write vector copy code
    memcpy(p_y, p_x, BUF_SIZE * sizeof(float));
    
    // Step 5. CUDA kernel call
    
    // Step 4. CUDA memcpy (Device -> Host)
    cudaMemcpy(p_y, d_y, BUF_SIZE * sizeof(float), cudaMemcpyDeviceToHost);
    
    float sum = 0.f;
    for (int i = 0; i < BUF_SIZE; i++) {
        sum += p_y[i] - p_x[i];
    }
    printf("Diff. Sum: %f", sum);
    
    // Step 2. CUDA buffer free
    cudaFree(d_x);
    cudaFree(d_y);
    
    free(p_x);
    free(p_y);
}

In [ ]:
!make kernel_call && ./kernel_call